# Export data for FEA
Use MI exporters to export records for use with FEA, CAE or CAD packages.

## Find exporters
Check which exporters are available for a specific table.

In [1]:
from GRANTA_MIScriptingToolkit import granta as mpy

mi = mpy.connect('http://localhost/mi_servicelayer', autologon=True)
db = mi.get_db(db_key='MI_Training')
table = db.get_table('Design Data')

exporters_in_table = table.get_available_exporters(package='ANSYS Workbench')

print("\nOutput of available ANSYS Workbench Exporters")
for exporter in exporters_in_table:
    print("{0} ({1}) - {2}".format(exporter.name, exporter.package, exporter.description))


Output of available ANSYS Workbench Exporters
458E9A7E-C268-4ED0-9CC1-FF7438521B4F (ANSYS Workbench) - Exports linear, temperature-independent, isotropic data to the Ansys Workbench format
CE8DCFA2-B3EE-4545-8D3E-82810FA92AFC (ANSYS Workbench) - Exports linear, temperature-dependent, isotropic data to the Ansys Workbench format
4B0B1EA3-8760-43DF-8060-2C79CA471D4C (ANSYS Workbench) - Exports linear, temperature-independent, isotropic with simple failure data to the Ansys Workbench format


Check which exporters are applicable to a specific record.

In [2]:
rec = table.search_for_records_by_name('250 Grade Maraging, Maraged at 900F, Plate, Thickness: 0.1875 to 0.251 in, AMS 6520, S basis')[0]
applicable_exporters = rec.get_available_exporters()
print("\nOutput of exporters for 250 Grade Maraging steel:")

for exporter in applicable_exporters:
    print("{0} ({1}) - {2}".format(exporter.name,
                                   exporter.package,
                                   exporter.description))


Output of exporters for 250 Grade Maraging steel:
71CE1C21-FDEA-4119-B481-81BDC41BD900 (Abaqus 6) - Exports temperature dependent, isotropic data to the Abaqus format.
5C560880-4FD3-4E5C-992B-4B6CEF6A055A (Abaqus 6) - Exports temperature independent, isotropic data to the Abaqus 6 format.
911AF055-B388-439A-8AF6-EB18480E2D80 (Abaqus 6) - Linear, temperature-independent, isotropic, simple failure
3AE2BEA5-B1DB-45D3-A431-48915B8D1317 (Abaqus 6) - Linear, temperature-independent, isotropic, simple failure with thermal expansion
722E5C46-3633-4B72-BF93-74E8112C20C3 (Abaqus 6) - Exports temperature dependent, isotropic data to the Abaqus 6 format.
B653C213-8BEB-42A7-8512-5F340EEBFAB4 (Abaqus 6) - Exports temperature independent, isotropic data to the Abaqus 6 format.
458E9A7E-C268-4ED0-9CC1-FF7438521B4F (ANSYS Workbench) - Exports linear, temperature-independent, isotropic data to the Ansys Workbench format
CE8DCFA2-B3EE-4545-8D3E-82810FA92AFC (ANSYS Workbench) - Exports linear, temperatur

## Working with parameters
Some exporters support parameters. The exported parameters have default values but they can also be set manually.

Get the required parameters for an exporter.

In [3]:
exporter_to_use = rec.get_available_exporters(package="Abaqus 6",
                                              model="Linear, temperature-dependent, isotropic, thermal, plastic")[0]

parameters_required = exporter_to_use.get_parameters_required_for_export([rec])
print(parameters_required)

{'Time': <ParameterDefinition name: Time, type: Unrestricted numeric, unit: hr>}


Set the parameter values and perform the export to obtain the material card for use with FEA software. Provide one or more records to the exporter, up to the maximum number specified in the exporter configuration file. The records will be exported to the same material card.

In [4]:
parameter_values = {"Time": 100.0}
for parameter_name in parameters_required.keys():
    parameters_required[parameter_name].value_for_exporters = parameter_values[parameter_name]

material_card = exporter_to_use.run_exporter([rec], parameter_defs=parameters_required)
print(material_card)

**
**Granta Material Name: 250 Grade Maraging, Maraged at 900F, Plate, Thickness: 0.1875 to 0.251 in, AMS 6520, S basis
**Model Type: Linear, temperature-dependent, isotropic, thermal, plastic
**Unit System: SI (Consistent)
**Export User: DOMAIN\username
**Export DateTime: 2021-07-15T09:51:26.8735379+00:00
**Database Name: MI Training
**Table Name: Design Data
**Material Record History Id: 20673
**Density: 7.92e+3 kg/m^3
**Thermal conductivity with temperature: 37 datapoints W/m.°C
**Thermal expansion coefficient with temperature: 103 datapoints strain/°C
**Tensile strength, yield with temperature: 0 datapoints Pa
**Tensile modulus with temperature: 0 datapoints Pa
**Poissons ratio: 3.10e-1
**Thermal diffusivity with temperature: [No Value]
**
**Comments
**
**LinearIsoTempThermalPlastic.xslt: Transformed CAE material model data
**(Linear, temperature-dependent, isotropic, thermal, plastic) to Abaqus 6 format.
**
**
*MATERIAL, NAME = A250_Grade_Maraging_Maraged_at
**
*DENSITY
7.92e+3
**

Save the exported data using the settings provided in the .exp file. The filename, extension, and file encoding scheme are left as their defaults.

In [5]:
path_to_save = "./"
file_name = 'Example_Export'
exporter_to_use.save(path_to_save, file_name="Example_Export")
file_extension = exporter_to_use.default_file_extension
print("Exporter output saved to \"{}{}.{}\"".format(path_to_save, file_name, file_extension))

Exporter output saved to "./Example_Export.inp"
